# Training TensorFlow Trading Bot

In [1]:
import os

data_dpath = os.path.join(os.getcwd(), "data")
models_dpath = os.path.join(os.getcwd(), "models")
os.chdir(os.path.pardir)

## Model Configuration

In [2]:
import tensorflow as tf

INIT_LEARNING_RATE = 0.001
BATCH_SIZE = 64
EPOCHS = 100
CLASSIFICATION_THRESHOLD = 0.35

METRICS = [
    tf.keras.metrics.BinaryAccuracy(
        name='accuracy', threshold=CLASSIFICATION_THRESHOLD
    ),
    tf.keras.metrics.Precision(
        name="precision", thresholds=CLASSIFICATION_THRESHOLD,
    )
]

In [3]:
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.initializers import Constant
from tensorflow.python.keras.optimizers import adam_v2

def make_classifier(num_features: int, outputs: np.ndarray, init_learning_rate: float = INIT_LEARNING_RATE,
        metrics: list = METRICS):

    positives = np.sum(outputs)
    negatives = outputs.shape[0] - positives
    bias_initializer = Constant(np.log([positives/ negatives]))

    # Layers
    input_layer = Input(shape=(num_features))
    dense_layer = Dense(units=512, activation="relu")(input_layer)
    dense_layer = Dense(units=128, activation="relu")(dense_layer)
    dense_layer = Dense(units=32,  activation="relu")(dense_layer)
    output_layer = Dense(units=1, activation="sigmoid", bias_initializer=bias_initializer) \
            (dense_layer) # Binary output

    model = keras.Model(input_layer, output_layer)
    model.compile(optimizer=adam_v2.Adam(init_learning_rate), loss='binary_crossentropy', metrics=metrics)

    return model

## Loading Datasets

In [4]:
import pickle

with open(os.path.join(data_dpath, "data_pipeline.pickle"), "rb") as file:
    data_pipeline = pickle.load(file)
    
    candle_buffers = data_pipeline.get("candle_buffers")
    features_compiler = data_pipeline.get("features_compiler")

observations = np.load(os.path.join(data_dpath, "observations.npy"))
long_signals = np.load(os.path.join(data_dpath, "long_signals.npy"))
short_signals = np.load(os.path.join(data_dpath, "short_signals.npy"))

## Making and Training Trading Engines

In [5]:
long_classifier = make_classifier(features_compiler.get_num_features(), long_signals)
short_classifier = make_classifier(features_compiler.get_num_features(), short_signals)

In [6]:
import pandas as pd
o = pd.DataFrame(observations)

In [7]:
long_classifier.fit(
    x=observations[:long_signals.shape[0]],
    y=long_signals,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.05,
    verbose=True
)

Epoch 1/10
103/103 [==============================] - 1s 4ms/step - loss: 1.7034 - accuracy: 0.6894 - precision: 0.4804 - val_loss: 6.8890 - val_accuracy: 0.7779 - val_precision: 0.6246
Epoch 2/10
103/103 [==============================] - 0s 2ms/step - loss: 0.2367 - accuracy: 0.8088 - precision: 0.6804 - val_loss: 5.9561 - val_accuracy: 0.8300 - val_precision: 0.7205
Epoch 3/10
103/103 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.8428 - precision: 0.7442 - val_loss: 7.8875 - val_accuracy: 0.8538 - val_precision: 0.7651
Epoch 4/10
103/103 [==============================] - 0s 2ms/step - loss: 0.1307 - accuracy: 0.8634 - precision: 0.7831 - val_loss: 9.1590 - val_accuracy: 0.8707 - val_precision: 0.7982
Epoch 5/10
103/103 [==============================] - 0s 3ms/step - loss: 0.1652 - accuracy: 0.8771 - precision: 0.8104 - val_loss: 8.2059 - val_accuracy: 0.8801 - val_precision: 0.8169
Epoch 6/10
103/103 [==============================] - 0s 2ms/step - lo

In [8]:
long_classifier.save(os.path.join(models_dpath, "training", "long_trading_engine"))

INFO:tensorflow:Assets written to: c:\Users\jacob\repositories\Trading\bitfinex_daytrading\models\training\long_trading_engine\assets


In [ ]:
short_classifier.fit(
    x=observations[:short_signals.shape[0]],
    y=short_signals,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.05,
    verbose=True
)

In [ ]:
short_classifier.save(os.path.join(models_dpath, "training", "short_trading_engine"))